In [1]:
import coremlv2 as core
core._init_ml()
core.os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import copy
core.tf.get_logger().setLevel('INFO')

import importlib
importlib.reload(core)

<module 'coremlv2' from 'C:\\Users\\Jonathan Raditya\\documents\\#PROJECT\\idx\\coremlv2.py'>

In [2]:
def set_nontrainable_except_dense(model):
    for i, layer in enumerate(model.layers):
        if not core.re.search('dense', layer.name):
            # print(i, layer.name, ': Set to non-trainable.')
            layer.trainable = False
            
def make_multiple_shot_input(idx_df, df, input_c, recurrent, train_mean, train_std):
    '''Make multiple shot input dataset
    
    Params:
        df_idx: idx available trading date
        df: dataframe correspond to specified ticker
        
    Returns:
        input_array
        available_date
    '''
    available_date = []
    first = True
    for i, selected_date in enumerate(idx_df['time']):
        try:
            sel_input_index = df.loc[df['time'] == selected_date].index[0]
        except IndexError:
            continue

        available_date.append(selected_date)

        # Structure and normalize 1-shot input data
        input_data = (df[sel_input_index - recurrent + 1: sel_input_index + 1][input_c] - train_mean) / train_std
        input_data_np = input_data.to_numpy()
        input_data_np = input_data_np.reshape(1, input_data_np.shape[0], input_data_np.shape[1])

        if first:
            multiple_shot_input_data = copy.deepcopy(input_data_np)
            first = False
        else:
            multiple_shot_input_data = core.np.vstack((multiple_shot_input_data, input_data_np))
    return multiple_shot_input_data, available_date

# Trying to train from database the column have been selected through 'correlation test'
Result: actually the performance very bad

In [ ]:
# INFO: Ater trying to build dnn over this dataset, it found
# that even at baseline (LSTM+D+D) the model still hover
# at 0.3631 huber_loss. Possible dataset fault.
ticker = 'BBCA'
db_ver = '3'
DB_ROOT_PATH = f'{ROOT_PATH}db/v{db_ver}/'
v2_db = f'{DB_ROOT_PATH}idx_{ticker}.db'
v2_db_conn = core.sqlite3.connect(v2_db)

df = core.restore_splitdf(v2_db_conn)
df = df.fillna(0)

In [ ]:
epochs = 10

coremlv2.tf.keras.backend.clear_session()
model = model_132(recurrent, train_df_input)
history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs, verbose=1)

model.summary()

# Aim: get best val performance: simple model
Result: this is the current best possible way to construct the model

In [316]:
# Bring back simple dataset with 70+ features
# and find the best possible architecture.
# Build up additional features based on 
# performance of this benchmark dataset.

# After digging a little more, it turns out that
# `candle*` single handedly worsening the performance.
ticker = 'PGAS'
ROOT_PATH = './'
DB_PATH = f'{ROOT_PATH}idx_indicators.db'
db_conn = core.sqlite3.connect(DB_PATH)
df = core.pd.read_sql(f'select * from {ticker}', db_conn)
df = df.fillna(0)
df = df[-500:] # select data from last 5 years (5*250td)
df = df.reset_index(drop=True)

In [317]:
recurrent = 5
label_column = 'close_EMA3_G_s2'

train_df, test_df, train_mean, train_std = core.split_traintest(df, split=0.8)
train_df_input = core.drop_unallowed_columns(train_df)
test_df_input = core.drop_unallowed_columns(test_df)

input_c = [column for column in train_df_input.columns if column != label_column]

train_dataset = core.TimeseriesGenerator(train_df[input_c], train_df[label_column], length=recurrent, batch_size=128, shuffle=False)
test_dataset = core.TimeseriesGenerator(test_df[input_c], test_df[label_column], length=recurrent, batch_size=128)

In [ ]:
epochs = 20

core.tf.keras.backend.clear_session()
model = core.model_switcher('188', recurrent, train_df_input)
model.summary()

history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs, verbose=1)

# Try transfer learning from whole idx data by freezing conv and recurrent layer
Result: Work well enough

In [40]:
# Bring back simple dataset with 70+ features
# and find the best possible architecture.
# Build up additional features based on 
# performance of this benchmark dataset.

# After digging a little more, it turns out that
# `candle*` single handedly worsening the performance.
ticker = 'BBCA'
ROOT_PATH = './'
DB_PATH = f'{ROOT_PATH}idx_indicators.db'
db_conn = core.sqlite3.connect(DB_PATH)
df = core.pd.read_sql(f'select * from {ticker}', db_conn)
df = df.fillna(0)
df = df[-500:] # select data from last 5 years (5*250td)
df = df.reset_index(drop=True)

In [41]:
label_column = 'close_EMA3_G_s2'
ROOT_PATH = core.os.path.join('./')
interim_model = core.os.path.join(ROOT_PATH, 'models/explore_idx/v2/interim/model/')
model_path = core.os.path.join(interim_model, label_column)
model = core.tf.keras.models.load_model(model_path)
set_nontrainable_except_dense(model)

recurrent = model.layers[0].output.shape[1]
model.summary()

0 conv1d : Set to non-trainable.
1 lstm : Set to non-trainable.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 5, 32)             6176      
                                                                 
 lstm (LSTM)                 (None, 1000)              4132000   
                                                                 
 dense (Dense)               (None, 32)                32032     
                                                                 
 dense_1 (Dense)             (None, 8)                 264       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 4,170,481
Trainable params: 32,305
Non-trainable params: 4,138,176
___________________________________________

In [43]:
train_df, test_df, train_mean, train_std = core.split_traintest(df, split=0.8)
train_df_input = core.drop_unallowed_columns(train_df)
test_df_input = core.drop_unallowed_columns(test_df)

input_c = [column for column in train_df_input.columns if column != label_column]

train_dataset = core.TimeseriesGenerator(train_df[input_c], train_df[label_column], length=recurrent, batch_size=128, shuffle=False)
test_dataset = core.TimeseriesGenerator(test_df[input_c], test_df[label_column], length=recurrent, batch_size=128)

epochs = 2

history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs, verbose=1)

Epoch 1/2
4/4 [==============================] - 4s 417ms/step - loss: 0.3279 - mean_squared_error: 0.8346 - root_mean_squared_error: 0.9136 - val_loss: 0.2325 - val_mean_squared_error: 0.5786 - val_root_mean_squared_error: 0.7607
Epoch 2/2
4/4 [==============================] - 1s 164ms/step - loss: 0.3198 - mean_squared_error: 0.8113 - root_mean_squared_error: 0.9007 - val_loss: 0.2355 - val_mean_squared_error: 0.5883 - val_root_mean_squared_error: 0.7670


# Try to run whole data training in pc
Result: Not possible with current computing power. There are 4 columns, for each column @10 epochs it getting about 15hrs, without prior knowledge that this method would work.

In [8]:
# Using whole idx data to perform universal training
# After the train has finished, freeze all layers
# up to RNN, and replace the dense. 
# Retrain using last 5 years data every stock in idx
ROOT_PATH = './'
DB_PATH = f'{ROOT_PATH}idx_merges.db'
db_conn = coremlv2.sqlite3.connect(DB_PATH)
df = coremlv2.pd.read_sql(f'select * from idx', db_conn)
df = df.fillna(0)

In [ ]:
# Model Engineering
label_column = 'close_EMA3_G_s2'
label_columns = ['close_EMA3_G_s1','close_EMA10_G_s2','close_EMA3_G_s2','close_EMA10_G_s1']
recurrent = 60

train_df, test_df, train_mean, train_std = coremlv2.split_traintest(df, split=0.8)
train_df_input = drop_unallowed_columns(train_df)
test_df_input = drop_unallowed_columns(test_df)

input_c = [column for column in train_df_input.columns if column != label_column]

epochs = 10
model = model_132(recurrent, train_df_input)
histories = []
for label_column in label_columns:
    coremlv2.tf.keras.backend.clear_session()
    train_dataset = coremlv2.TimeseriesGenerator(train_df[input_c], train_df[label_column], length=recurrent, batch_size=64, shuffle=False)
    test_dataset = coremlv2.TimeseriesGenerator(test_df[input_c], test_df[label_column], length=recurrent, batch_size=64)
    history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs, verbose=1)
    histories.append(history)

# Try transfer learning at scale
Freezing conv and recurrent layer

retrain v2:
- fix train output mean/std yang belum diganti referensinya ke label_column
- change mkdir -> makedirs
- perbaikan timpa predicted value (yang harusnya observed_date)

In [13]:
ROOT_PATH = core.os.path.join('./')
DB_PATH = core.os.path.join(ROOT_PATH, 'idx_indicators.db')
explore_version = 2
retrain_version = 2
interim_model = core.os.path.join(ROOT_PATH, f'models/explore_idx/v{explore_version}/interim/model/')
interim_retrained = core.os.path.join(ROOT_PATH, f'models/explore_idx/v{explore_version}/retrained/v{retrain_version}/')
label_columns = ['close_EMA3_G_s2', 'close_EMA3_G_s1', 'close_EMA10_G_s2']
batch_size = 128
train_shuffle = False
data_limit = 5 # last x years
epochs = 2

db_conn = core.sqlite3.connect(DB_PATH)
tickers = core.stock_list_100_highestrank_and_availability()

# Fetch idx df for backtest
start_date = core.datetime.datetime(2021,1,1)
end_date = core.datetime.datetime(2021,8,31)
idx_df = core.pd.read_sql('select * from IDX', db_conn)
idx_df = idx_df.loc[(idx_df['time'] >= core.pd.Timestamp(start_date).value) & (idx_df['time'] <= core.pd.Timestamp(end_date).value)]
idx_df = idx_df[['time']]

histories = {}
histories_path = core.os.path.join(interim_retrained, 'histories/')
if not core.os.path.exists(histories_path):
    core.os.makedirs(histories_path, mode=0o777)

count = 0
for ticker in tickers:
    histories[ticker] = {}
    
    df = core.pd.read_sql(f'select * from {ticker}', db_conn)
    df = df.fillna(0)
    df = df[-250 * data_limit:]
    df = df.reset_index(drop=True)
    
    train_df, test_df, train_mean, train_std = core.split_traintest(df, split=0.8)
    train_df_input = core.drop_unallowed_columns(train_df)
    test_df_input = core.drop_unallowed_columns(test_df)
    
    for label_column in label_columns:
        tick = core.datetime.datetime.now()
        histories[ticker][label_column] = {}
        histories[ticker][label_column]['backtest'] = {}
        model_path = core.os.path.join(interim_model, label_column)
        model = core.tf.keras.models.load_model(model_path)
        set_nontrainable_except_dense(model)
        recurrent = model.layers[0].output.shape[1]
        
        input_c = [column for column in train_df_input.columns if column != label_column]
        
        train_dataset = core.TimeseriesGenerator(train_df[input_c], train_df[label_column], length=recurrent, batch_size=batch_size, shuffle=train_shuffle)
        test_dataset = core.TimeseriesGenerator(test_df[input_c], test_df[label_column], length=recurrent, batch_size=batch_size)
        
        history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs, verbose=0)
        
        ## Backtest ##
        # Fetch observed data
        offset = int(label_column[-1])
        obs_output_c = df.loc[((df['time'] >= core.pd.Timestamp(start_date).value) & (df['time'] <= core.pd.Timestamp(end_date).value))][label_column]
        obs_change_plus1 = df.loc[((df['time'].shift(offset) >= core.pd.Timestamp(start_date).value) & (df['time'].shift(offset) <= core.pd.Timestamp(end_date).value))]['change']
        
        # Skip if no records exists during selected date
        if len(obs_output_c) == 0:
            pass
        else:
            # Make input dataset
            train_mean_sliced = list(train_mean[train_df_input.columns].values)
            train_std_sliced = list(train_std[train_df_input.columns].values)
            train_output_mean = train_mean[label_column]
            train_output_std = train_std[label_column]
            multiple_shot_input_data, available_date = make_multiple_shot_input(idx_df, df, input_c, recurrent, train_mean_sliced, train_std_sliced)
            
            predicted = model.predict(multiple_shot_input_data)
            predicted_unnormalize = predicted * train_output_std + train_output_mean
            
            histories[ticker][label_column]['backtest']['predicted_value'] = predicted_unnormalize.tolist()
            histories[ticker][label_column]['backtest']['predicted_date'] = available_date
            histories[ticker][label_column]['backtest']['observed_value'] = (obs_output_c.to_numpy()).tolist()
            histories[ticker][label_column]['backtest']['observed_date'] = available_date
            histories[ticker][label_column]['backtest']['start_date'] = start_date.strftime('%Y%m%d')
            histories[ticker][label_column]['backtest']['end_date'] = end_date.strftime('%Y%m%d')
            
            if offset > 1:
                histories[ticker][label_column]['backtest'][f'change_{offset}'] = (obs_change_plus1.to_numpy()).tolist()
                histories[ticker][label_column]['backtest'][f'change_{offset}_date'] = available_date 
        
        # Store to histories
        histories[ticker][label_column]['params'] = history.params
        histories[ticker][label_column]['history'] = history.history
        histories[ticker][label_column]['train_mean'] = train_mean_sliced
        histories[ticker][label_column]['train_std'] = train_std_sliced
        
        # Save model
        model.save_weights(core.os.path.join(interim_retrained, f'weights/{ticker}/{label_column}/'))
        model.save(core.os.path.join(interim_retrained, f'model/{ticker}/{label_column}/'))
        
        # Save histories to drive
        with open(core.os.path.join(histories_path, 'histories.json'), 'w') as f:
            core.json.dump(histories, f)
            
        count+=1
        tock = core.datetime.datetime.now()
        print(f'Current progress: {count}/{len(tickers) * len(label_columns)} {ticker} {label_column}, time: {tock - tick}')

INFO:tensorflow:Assets written to: ./models/explore_idx/v2/retrained/v2/model/BBRI/close_EMA3_G_s2/assets


INFO:tensorflow:Assets written to: ./models/explore_idx/v2/retrained/v2/model/BBRI/close_EMA3_G_s2/assets


Current progress: 1/300 BBRI close_EMA3_G_s2, time: 0:00:32.319024


INFO:tensorflow:Assets written to: ./models/explore_idx/v2/retrained/v2/model/BBRI/close_EMA3_G_s1/assets


INFO:tensorflow:Assets written to: ./models/explore_idx/v2/retrained/v2/model/BBRI/close_EMA3_G_s1/assets


...

INFO:tensorflow:Assets written to: ./models/explore_idx/v2/retrained/v2/model/CSAP/close_EMA3_G_s1/assets


Current progress: 299/300 CSAP close_EMA3_G_s1, time: 0:00:37.356533


INFO:tensorflow:Assets written to: ./models/explore_idx/v2/retrained/v2/model/CSAP/close_EMA10_G_s2/assets


INFO:tensorflow:Assets written to: ./models/explore_idx/v2/retrained/v2/model/CSAP/close_EMA10_G_s2/assets


Current progress: 300/300 CSAP close_EMA10_G_s2, time: 0:00:37.072571


In [ ]:
predicted_unnormalize.tolist()

In [36]:
history.params

{'verbose': 1, 'epochs': 2, 'steps': 4}

In [38]:
history.history

{'loss': [0.37176406383514404, 0.35951536893844604],
 'mean_squared_error': [0.9626070857048035, 0.9239258766174316],
 'root_mean_squared_error': [0.9811254143714905, 0.9612106084823608],
 'val_loss': [0.2879769802093506, 0.2936418652534485],
 'val_mean_squared_error': [0.6900486350059509, 0.704744279384613],
 'val_root_mean_squared_error': [0.8306916356086731, 0.8394904732704163]}

In [54]:
train_mean['close_EMA3_G_s1']

1.494290644748844

In [53]:
label_columns = ['close_EMA3_G_s2', 'close_EMA3_G_s1', 'close_EMA10_G_s2', 'close_EMA10_G_s1']
for label_column in label_columns:
    print(label_column[-1])

2
1
2
1


In [55]:
start_date = core.datetime.datetime(2021,1,1)

In [51]:
core.pd.Timestamp(1635465600000000000)

Timestamp('2021-10-29 00:00:00')

In [57]:
start_date.strftime('%Y%m%d')

'20210101'